In [1]:

!pip install pyspark==3.1.2

In [2]:
from pyspark import SparkConf, SparkContext
from google.colab import files
import csv
conf = SparkConf().setAppName("customer_orders")
sc = SparkContext(conf=conf)

In [4]:
uploaded = files.upload()

Saving Asssignment2_customer-orders.csv to Asssignment2_customer-orders (3).csv


In [5]:
input_file_path = next(iter(uploaded))
data = sc.textFile(input_file_path)
#Filter out header row from parsing
header = data.first()
data = data.filter(lambda line: line != header)

In [7]:
def parse_line(line):
    fields = line.split(",")
    customer_id = int(fields[0])
    item_purchased = int(fields[1])
    purchase_price = float(fields[2])
    return (customer_id, item_purchased, purchase_price)

parsed_data = data.map(parse_line)

In [14]:
# Step 1: Calculate the total purchase amount for each customer
customer_totals = parsed_data.map(lambda x: (x[0], x[2])) \
    .reduceByKey(lambda x, y: x + y)

# Step 2: Sort the customers by their total purchase amount in descending order
sorted_customers = customer_totals.sortBy(lambda x: x[1], ascending=False)

# Step 3: Take the top 5 customers
top_5_customers = sorted_customers.take(5)

# Print the top 5 customers
for customer in top_5_customers:
    print("Customer ID:", customer[0], "Total Purchase Amount:", customer[1])


Customer ID: 28 Total Purchase Amount: 309717.5700000003
Customer ID: 73 Total Purchase Amount: 308126.83999999997
Customer ID: 52 Total Purchase Amount: 307093.32000000024
Customer ID: 33 Total Purchase Amount: 306820.3300000003
Customer ID: 58 Total Purchase Amount: 305663.10999999975


In [15]:
top_10_customers = sorted_customers.take(10)

In [17]:
# Find the item purchased by each customer
items_purchased_by_top_10 = parsed_data.filter(lambda x: x[0] in [customer[0] for customer in top_10_customers]) \
    .map(lambda x: (x[1], 1)) \
    .reduceByKey(lambda x, y: x + y)

# Find which item is purchased the most by the top 10 customers
most_purchased_item = items_purchased_by_top_10.sortBy(lambda x: x[1], ascending=False).first()


print("Item purchased the most among the top 10 customers:", most_purchased_item[0])
print("Count of the most purchased item:", most_purchased_item[1])






Item purchased the most among the top 10 customers: 1198
Count of the most purchased item: 94
